## 模型简介

>k近邻是一种监督算法，既可以用来解决分类问题，也可以用来解决回归问题。   
    1. 对于分类问题，对一个未知标签的样本找到最近的k个点，这些点称之为最近邻居，通过最近邻居投票计数得到该样本的类别。
    2. 对于回归问题，类似于上面的方案，只是这里得到的k个最近邻居的均值。
    
 最近邻算法跟多的是一种 `集思广益`思想的体现

最近邻涉及度量： 

|名称|定义|  
|--|--|
曼哈顿距离|一维空间两点间的距离
欧式距离|二维空间两点间的距离
切比雪夫距离|n（n趋于无穷）维空间两点间的距离
闵可夫斯基距离|上面距离的总称

### 手写nn分类器

#### 加载包及数据集

In [1]:
import numpy as np
from collections import Counter 
from sklearn import datasets
from sklearn.model_selection import train_test_split

#### 加载数据并切分训练集和测试集

In [2]:
data = datasets.load_digits()  #加载手写数字数据集
x = data.data  #数据
y = data.target  #标签
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
#划分数据集

#### knn算法实现并封装类

- 初始化参数；  
- fit训练模型，knn传入自身即可；
- 预测函数，计算测试集与训练集的欧氏距离，用投票的方法，挑选出其中票数最多的距离做为最近距离；
- 模型评价，使用预测的准确率作为评价；

In [3]:
class knnclassifier:
    def __init__(self, k):
        
        '''
        parmaters: 
        k: 控制最近距离个数 
        '''
        self.k = k 
        self.x = None
        self.y = None
    
    # 训练器
    def fit(self, x, y):
        
        self.x = x
        self.y = y

        return self

    # 估计器，返回模型预测结果
    def predict(self, evalate_model):
        
        y_predict = [self.evalate_model(x) for x in evalate_model]

        return np.array(y_predict)

    # 模型的计算过程
    def evalate_model(self, x_):
        
        # 计算距离
        d = [np.sqrt(np.sum((x - x_)**2)) for x in self.x]

        # 排序
        nearest = np.argsort(d)

        # 返回其中最近的k个
        top_y = [self.y[i] for i in nearest[:self.k]]

        # 使用计数器，投票选择票数最多的距离
        votes = Counter(top_y)
        return votes.most_common(1)[0][0]

    # 评估器
    def score(self, y_test, x_predict):

        y_predict = self.predict(x_predict)
        
        return sum(y_test == y_predict) / len(y_test)

In [4]:
Knn = knnclassifier(k=3)

In [5]:
f = Knn.fit(x_train, y_train)
p = Knn.predict(x_test)
Knn.score(y_test, x_test)

0.975

### sklearn 封装knn

    sklearn.neighbors 提供了 neighbors-based (基于邻居的) 无监督学习以及监督学习方法的功能  
    监督学习分为两种： classification （分类）针对的是具有离散标签的数据，regression （回归）针对的是具有连续标签的数据。   
    无监督的最近邻是许多其它学习方法的基础，尤其是 manifold learning (流行学习) 和 spectral clustering (谱聚类)。  


#### 无监督 


sklearn.neighbors.NearestNeighbors（n_neighbors = 5，radius = 1.0，algorithm ='auto'，leaf_size = 30，metric ='minkowski'，p = 2，metric_params = None，n_jobs = None，** kwargs) 

参数：	  
- n_neighbors ： int，optional（default = 5）  
    默认情况下kneighbors查询使用的邻居数。


- radius ： float，optional（默认值= 1.0）  
    默认情况下用于radius_neighbors 查询的参数空间范围。
  
  
- algorithm ： {'auto'，'ball_tree'，'kd_tree'，'brute'}，可选  
    用于计算最近邻居的算法：  
    'ball_tree' 将使用 BallTree  
    'kd_tree' 将使用 KDTree  
    'brute' 将使用蛮力搜索。  
    'auto' 将尝试根据传递给fit方法的值来确定最合适的算法。  
    注意：在稀疏输入上拟合将使用强力来覆盖此参数的设置。  


- leaf_size ： int，optional（默认值= 30）    
    叶子大小传递给BallTree或KDTree。这可能会影响构造和查询的速度，以及存储树所需的内存。最佳值取决于问题的性质。


- metric ： 字符串或可调用，默认为'minkowski'    
    用于距离计算的度量。可以使用scikit-learn或scipy.spatial.distance中的任何指标。
    
    
- p ： 整数，可选（默认= 2）    
    来自sklearn.metrics.pairwise.pairwise_distances的Minkowski度量的参数。当p = 1时，这相当于使用manhattan_distance（l1），并且对于p = 2使用euclidean_distance（l2）。对于任意p，使用minkowski_distance（l_p）。  


- metric_params ： dict，optional（默认=无）  
    度量函数的其他关键字参数。


- n_jobs ： int或None，可选（默认=无）  
    为邻居搜索运行的并行作业数。 None除非在joblib.parallel_backend上下文中，否则表示1 。 -1表示使用所有处理器。

In [6]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X) # 2个最近邻，ball_tree搜索
distances, indices = nbrs.kneighbors(X)

In [7]:
indices  #索引         

array([[0, 1],
       [1, 0],
       [2, 1],
       [3, 4],
       [4, 3],
       [5, 4]], dtype=int64)

In [8]:
distances #距离

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.41421356],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.41421356]])

In [9]:
nbrs.kneighbors_graph(X).toarray()
# 样本数A [i，j]被赋予连接i到j的边的权重

array([[1., 1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 1., 1.]])

    kneighbors(self[, X, n_neighbors, …])	找到一个点的k近邻点
    kneighbors_graph(self[, X, n_neighbors, mode])	计算X中点的k-邻居的（加权）图

另外，我们可以使用 KDTree 或 BallTree 来找最近邻。 这是上文使用过的 NearestNeighbors 类所包含的功能。 KDTree 和 BallTree 具有相同的接口 

KDTree(X, leaf_size=40, metric=’minkowski’, **kwargs)

BallTree(X, leaf_size=40, metric=’minkowski’, **kwargs)

#### 监督学习

分类  
sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=None, **kwargs)

回归  
sklearn.neighbors.KNeighborsRegressor(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=None, **kwargs)     
二者具有相似的api

**weights ： str或callable，可选（默认='uniform'）**  
用于预测的权重函数。可能的值：

'uniform'：均匀的重量。每个社区的所有积分均等。  
’distance'：重量点距离的倒数。在这种情况下，查询点的较近邻居将比远离的邻居具有更大的影响力。  
[callable]：一个用户定义的函数，它接受一个距离数组，并返回一个包含权重的相同形状的数组。

方法|用途
--|--
fit(self, X, y) | 使用X作为训练数据并使用y作为目标值来拟合模型
get_params(self[, deep]) |	获取此估算工具的参数。
kneighbors(self[, X, n_neighbors, …]) |	找到一个点的K邻居。
kneighbors_graph(self[, X, n_neighbors, mode])	|计算X中点的k-邻居的（加权）图
predict(self, X)	| 预测所提供数据的类标签（分类）/预测所提供数据的目标（回归）
predict_proba(self, X) |	测试数据X的返回概率估计。
score(self, X, y[, sample_weight])|	返回给定测试数据和标签的平均精度。(分类）/返回预测的确定系数R ^ 2。（回归）
set_params(self, \*\*params)|	设置此估算器的参数。

In [10]:
## 对比上面手写分类算法
from sklearn.neighbors import KNeighborsClassifier
clf=KNeighborsClassifier(n_neighbors=4,weights='distance') # 以距离的倒数为权重
clf.fit(x_train,y_train)
clf.predict(x_test)
clf.score(x_test,y_test)

0.975

#### 找到最优的k值和距离参数

In [11]:
### 遍历，找到最优的k值，调参
# 初始化
best_score = 0.0
best_k = 1

for k in range(1, 11):
    
    knn_clf = KNeighborsClassifier(n_neighbors=k)
    knn_clf.fit(x_train, y_train)
    score = knn_clf.score(x_test, y_test)
    
    if score > best_score:
        best_k = k
        best_score = score

print('best_score =', best_score)
print('best_k =', best_k)

best_score = 0.9777777777777777
best_k = 1


note:若找到最好值为10（在边界点），需要扩大参数搜索范围

In [12]:
#进一步找到最优的距离参数
# 初始化参数
best_score = 0.0
best_k = 1
best_method = ''

for method in ['uniform', 'distance']:
    for k in range(1, 11):
        
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights=method)
        knn_clf.fit(x_train, y_train)
        score = knn_clf.score(x_test, y_test)
        
        if score > best_score:
            best_k = k
            best_score = score
            best_method = method
            
print('best_score =', best_score)
print('best_k =', best_k)
print('best_method =', best_method)

best_score = 0.9805555555555555
best_k = 6
best_method = distance


使用sklearn封装的 GridSearchCV 同样可以进行上述调参操作   

sklearn.model_selection.GridSearchCV（estimator，param_grid，scoring = None，n_jobs = None，iid ='warn'，refit = True，cv ='warn'，verbose = 0，pre_dispatch ='2 * n_jobs'，error_score ='raise-deprecating'，return_train_score = False )

[参数:](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)

如果将n_jobs设置为大于1的值，则会为网格中的每个点复制数据（而不是n_jobs次）。如果单个作业花费的时间很少，则可以提高效率，但如果数据集很大且内存不足，则可能会出错。这种情况下的解决方法是设置pre_dispatch。然后，内存仅复制pre_dispatch多次。 pre_dispatch的合理值是2 * n_jobs。

In [13]:
param_grid = [{
    'weights': ['uniform'],
    'n_neighbors': [i for i in range(1, 11)]
}, {
    'weights': ['distance'],
    'n_neighbors': [i for i in range(1, 11)],
    'p': [i for i in range(1, 6)]
}]

knn_clf = KNeighborsClassifier()
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(knn_clf, param_grid)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [14]:
grid_search.best_estimator_ #最好估计器

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [15]:
grid_search.best_score_

0.9909577042198995